# 1. Optimization of deep learning parameters (딥러닝 파라미터의 최적화)
- 완전 연결층, 합성곱 신경망, LSTM층을 설계하고 구현하기 위해서는 다양한 파라미터의 값을 지정해야 함
1) 데이터 준비
2) 딥너링 모델 생성
3) 학습
4) 검증

In [1]:
# 데이터 준비
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers

data_no = pd.read_csv("ptbdb_normal.csv")
data_ab = pd.read_csv("ptbdb_abnormal.csv")
data_no = np.array(data_no)
data_ab = np.array(data_ab)

nTrain = 3000
nTest = 1000
X_train = np.concatenate((data_no[:nTrain, :], data_ab[:nTrain, :]), 0)
y_train = np.concatenate((np.zeros(nTrain, ), np.ones(nTrain, )), 0)
X_test = np.concatenate((data_no[nTrain:nTrain+nTest, :], data_ab[nTrain:nTrain+nTest, :]), 0)
y_test = np.concatenate((np.zeros(nTest, ), np.ones(nTest, )), 0)

y_test = to_categorical(y_test)
y_train = to_categorical(y_train)
X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

In [2]:
# 딥러닝 모델 생성
model = Sequential()
model.add(layers.Conv1D(filters=16, kernel_size=3, input_shape=(X_train.shape[1], 1), activation="relu"))
model.add(layers.Conv1D(filters=16, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling1D(pool_size=3, strides=2))
model.add(layers.Conv1D(filters=32, kernel_size=3, input_shape=(X_train.shape[1], 1), activation="relu"))
model.add(layers.Conv1D(filters=32, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling1D(pool_size=3, strides=2))
model.add(layers.LSTM(16))
model.add(layers.Dense(units=2, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(learning_rate=0.01), metrics=["accuracy"])

In [3]:
# 학습
model.fit(X_train, y_train, epochs=50, batch_size=128, validation_split=0.2)

Epoch 1/50


38/38 [==============================] - 5s 37ms/step - loss: 0.6174 - accuracy: 0.6623 - val_loss: 0.8379 - val_accuracy: 0.5308
Epoch 2/50
38/38 [==============================] - 1s 19ms/step - loss: 0.5809 - accuracy: 0.7104 - val_loss: 0.8550 - val_accuracy: 0.4725
Epoch 3/50
38/38 [==============================] - 1s 18ms/step - loss: 0.5663 - accuracy: 0.7198 - val_loss: 0.8082 - val_accuracy: 0.5125
Epoch 4/50
38/38 [==============================] - 1s 18ms/step - loss: 0.5591 - accuracy: 0.7185 - val_loss: 0.6801 - val_accuracy: 0.6300
Epoch 5/50
38/38 [==============================] - 1s 18ms/step - loss: 0.5642 - accuracy: 0.7265 - val_loss: 0.7263 - val_accuracy: 0.5892
Epoch 6/50
38/38 [==============================] - 1s 18ms/step - loss: 0.5548 - accuracy: 0.7240 - val_loss: 0.7371 - val_accuracy: 0.5350
Epoch 7/50
38/38 [==============================] - 1s 18ms/step - loss: 0.5389 - accuracy: 0.7406 - val_loss: 0.9751 - val_accuracy: 0.4142
Epoch 8/50


In [4]:
# 검증
o = model.predict(X_test)
o = np.argmax(o, 1)
y_test = np.argmax(y_test, 1)
sum(np.equal(y_test, o) / len(y_test))

63/63 [==============================] - 1s 5ms/step


0.9539999999999503

## 1-1. 합성곱층의 파라미터
- model.add(layers.Conv1D(filters=16, kernel_size=3, activation="relu"))
- model.add(layers.MaxPooling1D(pool_size=3, strides=2))
- 모델에 합성곱층과 풀링충을 추가하고 파라미터를 설정하는 코드 
- 간단한 코드이지만 합성 곱층은 LSTM 층이나 완전 연결층에 비해 설정해 주어야 할 부분이 비교적 많은 편
1) 필터 수
- filters=16
- 필터는 합성곱을 수행하는 가중치의 세트
- 필터의 개수가 많아질수록 보다 다양한 특징을 추출할 수 있음
- 복잡한 데이터를 인식하기 위해서는 다양한 특징을 추출하여 정보량을 늘리는 것은 꼭 필요하기 때문에, 필터 수를 충분히 크게 지정할 필요가 있음
- 일반적으로 필터 수가 클수록 올바르게 학습된 네트워크의 성능은 향상되지만, 필터가 올바르게 작동하기 위한 필터 내부의 가중치들이 올바르게 학습되어야 하고, 이런 네트워크 학습이 더욱 어려워짐
- 이 문제를 해결하기 위한 기본 전략은 일단 과도하게 많은 수의 필터를 사용하여 학습을 시도한 후, 크기를 조금씩 줄여나가는 것
- 다른 전략은 여러 개의 합성곱층을 사용하면서, 필터 수를 조금씩 늘려가는 것
- 합성곱층에서 필터 수를 지정하는 코드의 예는 다음과 같음
- model.add(layers.Conv1D(filters=8, kernel_size=3, activation="relu"))
- model.add(layers.Conv1D(filters=16, kernel_size=3, activation="relu"))
- model.add(layers.Conv1D(filters=32, kernel_size=3, activation="relu"))
2) 커널 크기
- kernel_size = 3
- 커널 크기는 가중치 배열의 크기
- 커널 크기는 3(2차원인 경우 (3, 3))으로 쓰는 것이 보편적이긴 하지만, 상황에 따라 커널의 크기는 다양하게 바꿀 수 있음
- 커널 크기가 클수록 넓은 범위를 포괄할 수 있고, 작을수록 지엽적인 특징을 추출하는 필터가 만들어짐
- 일례로 커널 크기를 1로 설정하면, 출력 노드 1개 당 가중치의 개수가 하나로 고정되며, 출력은 단순히 공통의 가중치에 각 입력 노드의 값을 곱한 결과가 됨
3) 활성화 함수
- activation="relu"
- 활성화 함수는 각 층에서 발생하는 출력을 변형시키는 역할을 하며, 하이퍼볼릭 탄젠트 등은 출력을 특정 범위로 조정하기도 함
- 합성곱층과 완전 연결층에서 주로 사용하는 활성화 함수인 ReLU는 0보다 작은 출력을 모두 무시하는 역할을 함
- 이것은 부분적으로 딥러닝에서 사용되는 대부분의 데이터들이 0 또는 양수라는점에 기인하며, 가중치의 범위를 간접적으로 제한하여, 학습이 보다 쉽고 빠르게 이루어지게 하기 위한 것
- ReLU가 출력의 범위를 0 이상의 정수로 조정한다면, 0과 1 사이, -1과 1 사이의 값으로 출력을 조정하는 함수도 있음
- ReLU를 사용하면서도 비슷한 효과를 얻는 방법은 별도의 정규화 층을 추가하는 것
- 순환 신경망의 활성화 함수를 ReLU로 변경하여 성능을 향상시킬 수 있다는 의견이 있으나, 아직까지 Keras에서는 순환 신경망의 활성화 함수를 변경하는 방법을 별도로 제공하지는 않음
- Keras를 사용한 코딩에서 대부분의 활성화 함수들은 Activation 파라미터에 코드를 지정하여 사용할 수 있음
- 다만, Leaky ReLU는 고급 활성화 함수로 간주되어 별도의 네트워크층으로 구성하여야 함
- 활성화 함수를 별도의 층으로 구성할 때에는 합성곱층 등의 Activation 파라미터를 None으로 지정한 후, 해당 활성화 함수층을 추가
- 1차원 합성곱과 Leaky ReLU를 추가하는 코드는 다음과 같음
- model.add(layers.Conv1D(filters=16, kernel_size=3, activation=None))
- model.add(layers.LeakyReLU())
- 딥러닝 모델을 구성함에 있어 활성화 함수는 ReLU를 사용하는 것이 최근의 보편적인 흐름이며, 처음 모델을 구성할 때에는 ReLU를 사용하는 것이 최근의 보편적인 흐름이며, 처음 모델을 구성할 때에는 ReLU를 사용하여 모델을 구성하는 것을 추천
- 하지만 다른 딥러닝 파라미터와 마찬가지로, 여기에도 정답은 없고, 활성화 함수를 바꾸어 가며 학습/테스트를 진행하여 성능의 차이를 살펴봄
- model.add(layers.Conv1D(filters=32, kernel_size=3, activation="relu"))
4) 풀링층의 파라미터
- 풀링은 노드의 수, 즉 특징의 개수를 줄이기 위한 목적으로 사용
- 풀/stride의 크기가 클수록 특징의 개수가 대폭 줄기 때문에, 학습 난이도가 감소하지만, 동시에 정보 손실이 있을 수 있으므로 적절한 균형을 유지하도록하는 것이 중요
- 풀/stride의 크기는 네트워크의 깊이(개수)와도 관련이 있음
- 풀/stride의 크기가 크면, 네트워크층을 거쳐갈수록 입력 데이터의 크기가 빠르게 줄어들며, 반대의 경우에는 여러 층을 사용하여도 상대적으로 큰 크기를 유지할 수 있기 때문
- 예를 들어 16*16 크기의 이미지를 입력으로 받아, 풀/stride의 크기가 2인 풀링층을 세 번만 거치면 4개(2*2)의 노드가 남지만, 만약 크기가 4인 풀링층을 사용한다면 2개 이상의 풀링층은 사용할 수가 없게 됨
- 합성곱층과 풀링층이 하나의 세트로 사용되는 점을 생각해 보면, 이것은 네트워크를 구성함에 있어 제법 큰 제약이 됨
- 풀링층을 사용하면서도 여러 개의 합성곱층을 사용하는 '딥 네트워크'를 구성하기 위해, 합성곱층과 풀링층을 n:1의 비율로 구성하는 경우도 있음
- 합성곱 여러 개를 연속적으로 추가한 후, 풀링층을 한 번만 추가하는 것
- 다음은 2:1로 합성곱과 풀링층의 세트를 2개 연속으로 구성하는 코드
- model.add(layers.Conv1D(filters=32, kernel_size=3, activation="relu"))
- model.add(layers.Conv1D(filters=32, kernel_size=3, activation="relu"))
- model.add(layers.MaxPooling1D(pool_size=3, strides=2))
- model.add(layers.Conv1D(filters=32, kernel_size=3, activation="relu"))
- model.add(layers.Conv1D(filters=32, kernel_size=3, activation="relu"))
- model.add(layers.MaxPooling1D(pool_size=3, strides=2))
- 이미지 데이터를 인식하는 네트워크에서의 풀링은 MaxPooling이 사용되는데, 이것은 0에 가까울수록 정보가 적어지는 이미지 특성 때문
- MaxPooling을 사용하여 해당 풀에서 최댓값을 선택하므로 정보의 손실을 최소화하는 것
- 따라서 작은 값이 보다 중요한 정보를 지니고 있다면 MinPooling을, 모든 값이 중요한 상황이라면 성능을 높이기 위해 AveragePooling을 사용하여 테스트를 해 볼 필요가 있음
- MinPooling은 Keras에서는 지원하지 않으므로, 필요 시에는 입력 데이터를 반전하여 사용하거나 MinPooling을 자체적으로 구현해야 함
- Keras에서 제공하는 주요 풀링층은 Global 풀링층(입력받은 배치에 대해 전체 평균을 계산해 주는 층)과 Local 풀링층(일반적으로 사용)
- 풀링층의 또 다른 파라미터는 padding
- 이 파라미터의 값이 'valid'이면 패딩 없이 풀링을 수행하고, 'same'인 경우 stride가 1일 때를 기준으로 동일한 크기의 출력을 발생시키도록 제로-패딩을 한 후 결과를 계산
- 파라미터를 명시적으로 지정하지 않으면 기본값인 'valid'로 지정됨
- padding = "valid"

## 1-2. LSTM층의 파라미터
1) 노드 수
- LSTM층은 합성곱층에 비해 파라미터의 수가 적어, 파라미터의 지정이 비교적 단순
- LSTM에서 필수적인 